# Machine Learning 3rd Assignment Report

## PART I
### Theory Questions

#### 1
Only difference is the output type. Linear regression used to get continous values as output while logistic regression used to get categorical/binary/probabilistic values as output.

#### 2
Important difference is the type of the models.
Logistic regression is a discriminative model. Meaning it will use conditional probabilites to make predictions so learning involves conditional probabilities.
While naive bayes is a generative model. It will look for joint probabilites while learning. It involves all parameters at once so it is, well, generative.

#### 3
 - No. You need at least 1 hidden layer with at least two nodes. One hidden node will work as NAND and other works as AND gate. Combination of two gives XOR gate.
 - Yes. Neural networks with at least one hidden layer are universal approximators. They can represent any continous function. That includes logic ones too.
 - Probably no. There are a lot of activation function which you can use in tandem. There is no certain way we can write that equation.
 - Yes. Sigmoid function squashes values between 0 and 1. Just like softmax. That's why they are very popular as output layer activation functions for classification.
 
#### 4
What decides the shape of the model is the problem and dataset you have. If your problem is not complex you can actually see what model needs in order to solve the said problem. XOR for example requires a hidden layer because it is combination of two logic gates AND and NAND. Both of them can be implemented with single layer neural networks but to combine two we need to add a hidden layer with two nodes as I said before. <br>
But if your problem is complex, meaning you can't exactly build a logical model yourself you can always do some cross validation. Experience help a lot as well as knowing your data. <br>
A rule of thumb is not to go extremes. Too much or too little gives bad results. Try to find middle ground.<br>
Hidden layer count is should be especially small. As the layer count increases the complexity required from data also increases. If your model is too large for your data you will not get any proper result.

## Part II

### What
We are building a neural network and neural network interface to classify images of flowers. For that reason we are creating single and multi layer networks.

### How
Let's talk about what steps I followed for this problem. 

#### Structure
![file_structure](structure.png)
Firstly I had to create project structure. I created a python package for neural network interface and one for data. The data package was ready anyway so I just added methods to load the files. For neural network I wanted to create a dynamic network so I implemented a json loader method. From json template one can create different kinds of network similar to the ones in caffe.
#### Preparing data
Data is vectorized greyscale images of certain flower types. That handled most of the job. I took image vectors then squashed their intensity values between 0 and 1. Then I hot encoded their labels so I can classify them.
I did same thing for validation data so that I can validate as the model trained.
#### Regularization
I have tried few regularization methods like __l1__, __l2__ and __dropout__ regularization. None of them worked out well. Perhaps I failed to implement them or maybe model wasn't suitable for this kind of task. Anyway, I only left dropout since I implemented others pretty early on in a desperate attempt to improve the validation scores. When I saw that they actually worsened the situtation(underflows) I removed them and never looked back. At this point I believe it is just waste of time to reimplement them since they do nothing positive on the model. <br>
Let's talk about dropout specifically. It is the only regularization required a seperate method, meaning I could turn it on and off at will without much work. I added dropout method to the layers and it is called during forward pass after activation functions called. Neural network has a parameter called dropout_probabilty which used during dropout sequence. I used 0.5 mostly and never really tried anything else. During training with dropout, obviously, training scores drops significantly since half of the neurons not working during prediction while validation accuracy does not change a lot. 
#### Losses and Activations
I put every loss and activation function on seperate file called 'math_'. 
List of currently available activation functions:
 - __Sigmoid__: You can't go wrong with sigmoid. I polished it a lot so it is best implemented one out of the rest.
 - __Tanh__: Gives similar result to sigmoid. 
 - __Softmax__: Only used on output layer. Works pretty fine.
 - __ReLu__: Problem child. Because of dying relu problem you will quickly get all 0 outputs.
 - __Leaky ReLu__: In an attempt to solve the dying relu problem I added this. Never got it to work because of the overflows. Can't really care at this point.
 
I also have method for signum but I did not test it so you can't use it. <br>
List of currently available loss functions:

 - __Cross entropy__: First added loss function. Works just fine.
 - __Sum of squares__: Works ok, but did not tested it extensively.
 - __Mean squared error__: Implementation is derived from sum of squares and derivative calculated by me. So possibly does not work correctly.
 - __R Squared Error__: I added this last. While I was working with scikit learn I saw this and tried to implement it myself. Currently gives only negative output somehow so I suggest not using it.
 
Moral of the story, stick to the cross entropy.

### Analyses
Since my last assignment take a dip in score thanks to lack of analysis I worked hard on this one. It took several days for me to write down this part. So, I suggest you to get a cup of coffee.

#### <center>Analysis of activation functions</center>

|Parameter|Value|
|-|-|
|Hidden layer count|1|
|Hidden layer node count|40|
|Output activation|Softmax|
|Loss|Cross entropy|
|Epochs|300|
|Batch size|5|
|Learning rate bound|0.02 - 0.0005|
|Decay rate|0.0005|
|Dropout chance|0.0|
|Momentum parameter|1.0|


 - __Sigmoid:__
    > ![sigmoid](plots/layers/1_hidden_40_node_no_dropout.png)
    
 - __Tanh:__
    > ![tanh](plots/activations/tanh.png)  
    
 - __ReLu:__
    > ![relu](plots/activations/relu.png)
    
 - __Leaky Relu:__
    > ![leaky_relu](plots/activations/leaky_relu.png)
    
 > Looking at the results above:
  - __Looking at losses__ we see that sigmoid gives us best results. Loss converges lower values than the other functions. But if we look at relu it converges faster than sigmoid (starting from 450 then dropping to 200). But that means sigmoid gives us better initial prediction than relu so it is still best for current model.
  - __Looking at training accuracies__ for all models there is a lot of fluctuation and there is many reason for that including:
       * High learning rate
       * Lacking regularization
       * Low model size <br>
      how we going to fix is not problem of current analysis so I am going to leave it at that. But with training accuracies the best results came from the leaky relu. It peaked at much larger value than the others.
  - __Looking at validation accuracies__ we see that while leaky relu has the best training accuracy it actually starting to overfit at that peak point. While relu and tanh has immense fluctuations and dips the best average result came, again, from sigmoid. At it's last epoch training and validation accuracies was somewhat in unison meaning there is no overfit. But again accuracies are pretty low so we cannot make any concrete assumptions.
  
 > Overall the best results came from sigmoid. There is many reason for that:
     - First of all I worked longer on sigmoid implementation than the others. So it's coding quality might be higher that the others.
     - I tested it extensively since I, literally, build model on top of sigmoid so other activation functions might be giving incorrect results while sigmoid is surely giving right results.
     - Sigmoid is go to activation function when it comes to (binary) classification. While tanh also can be used for classification it's nature of including negative values gives it deep dips during training.

#### <center>Analysis of layer count</center>

|Parameter|Value|
|-|-|
|Hidden activation|Sigmoid|
|Output activation|Softmax|
|Loss|Cross entropy|
|Epochs|300|
|Batch size|5|
|Learning rate bound|0.02 - 0.0005|
|Decay rate|0.0005|
|Dropout chance|0.0|
|Momentum parameter|1.0|

- __Single layer:__
> ![single_layer](plots/layers/single_layer_no_dropout.png)

- __One hidden layer with 40 nodes:__
> ![10node1hidden](plots/layers/1_hidden_40_node_no_dropout.png)

- __One hidden layer with 100 nodes:__
> ![100node1hidden](plots/layers/1_hidden_100_node_no_dropout.png)

- __Two hidden layers with 40 and 40 nodes each:__
> ![40_40node2hidden](plots/layers/2_hidden_40_40_node_no_dropout.png)
 
> The graphs above selected because we have:
- Single layer sample
- One sample with single hidden layer
- One sample with single hidden layer but with increased node count
- One sample with two hidden layers with node counts same as the single hidden layer one <br>

> These tells us what we need to see:
- __Looking at loss__ we see that results are improving as we increse node counts but not the layer count. Single layer was enough to solve our problem but we may need to increase the number of nodes.
- __Looking at training accuracy__ we see again that we only need single layer but with larger number of nodes. Even then, results are close for all tests since overall accuracy is low.
- __Looking at validation accuracy__ we now see the best solution. While higher node counts leads to incresed training accuracy the gap between it and validation accuracy increases. As the layer count and node count increses training accuracy starting to overshoot the validation accuracy. That shows as our model is starting to overfit at that point.

From these results we can conclude that best possible model comes from _lower hidden layer count_ with _average node count._ Single layer simply wasn't enough for our data. 
  

#### <center>Analysis of loss functions</center>

|Parameter|Value|
|-|-|
|Hidden activation|Sigmoid|
|Hidden layer count|1|
|Hidden layer node count|40|
|Output activation|Softmax|
|Epochs|300|
|Batch size|5|
|Learning rate bound|0.02 - 0.0005|
|Decay rate|0.0005|
|Dropout chance|0.0|
|Momentum parameter|1.0|

- __Cross entropy:__
> ![cross_entropy](plots/layers/1_hidden_40_node_no_dropout.png)

- __Sum of squares error:__
> ![sum_of_squares](plots/losses/sum_of_squares.png)

- __Mean squared error:__
> ![mean_squared_error](plots/losses/mean_of_squares.png)
    Interesting. This is not that much different from sum of squares but it shows impressive difference. First of all that validation graph is the thing I would like to see in all others. A slow constant increse over epochs. Sadly it is a facade. I ran a test with 1,000 epochs and it was constant after that. Meaning it is the best model can do. Still cool.

- __r squared error:__
> ![r_squared](plots/losses/r_squared.png)
    Some notes: Perhaps I failed to implement R square, which is quite possible since I added it in a rush, loss always returns negative results. R squared can return negative results but for my model it shouldn't. Since it is already too late, I don't really care at this point. I am pretty sad tho.

> This is going to be one messy analysis.
- __Looking at the training accuracies__ cross entropy is simply best. Obvious learning, a bit less fluctuation and higher accuracy.
- __Looking at losses__ we can't tell which one exactly best since they every single one of them has different bounds. Especially r squared. Nearly all of them shows decrese in loss about 100 points (if that even have a meaning). R2 is pretty interesting, if we ignore the fact that it is negative it got pretty close to the 0.
- __Looking at validation accuracies__... oh god my eyes. R2 failed spectacularly. Cross entropy and sum of squares shows pretty close curves and less overfit overall.

> In my opinion __cross entropy__ is best out of all losses because:
 * First of all I implemented it first then implemented the rest of the model so it is tested more than any other loss.
 * Cross entropy is suggested loss function for multilabel classification.
 * Since softmax will always be on output layer as activation function(problem is a classification problem and softmax is best for that) and cross entropy is really common to use with softmax it is no brainer at this point.
 * Looking at results, also, we will see that there is less of fluctuation compared with other methods and validation score shows a similar curve with training score, again, compared with others.

#### <center>Analysis of batch sizes</center>

|Parameter|Value|
|-|-|
|Hidden activation|Sigmoid|
|Hidden layer count|1|
|Hidden layer node count|40|
|Output activation|Softmax|
|Loss function|Cross entropy|
|Epochs|300|
|Learning rate bound|0.02 - 0.0005|
|Decay rate|0.0005|
|Dropout chance|0.0|
|Momentum parameter|1.0|

I will be testing few batch sizes including the ones you have mentioned.

- __Batch size = 1:__
> ![batch_1](plots/batches/1.png)

- __Batch size = 2:__
> ![batch_1](plots/batches/2.png)

- __Batch size = 4:__
> ![batch_1](plots/batches/4.png)

- __Batch size = 16:__
> ![batch_1](plots/batches/16.png)

- __Batch size = 64:__
> ![batch_1](plots/batches/64.png)

- __Batch size = 128:__
> ![batch_1](plots/batches/128.png)

> Let's check out the graphs:
   - __Looking at the losses__ there is no major change in curve. The only change is in the bounds of the graph but that is given since loss is sum of errors not average of it (as per requested, I still believe average loss gives best overview but what do I know, right?). So the batch does not affect loss curve. Change is still amounts the same.
   - __Looking at the training accuracies__ we see a constant decrase over batch size. That is expected result for the batch method we asked for (will talk about it shortly). So what does this mean is if we increase batch size it is more likely to get stuck in local minima since our steps are shorter now. Also neither having low nor high batch size gives the best result. Best result, again, comes from average.
   - __Looking at the validation accuracies__ we see that batch size does __NOT__ affects the validation score by a huge margin.. Looking at graphs you might believe that there is a change but believe me, there is not many change. As the batch size increses validation score might seem like it fluctuates a lot but that because bound of graphs get closer together meaning we zoomed in to the model which exaggerates changes. The only change at very large number of batch size it drops average validation score for few percent.
   
> Few words about choosen batch method
We are suggested to use method below to calculate batch updates:
 * <p style='color:blue'>[Start of epoch]</p>
 * Split the dataset into equally sized batches. Batch size determined beforehand as hyper parameter.
 * Forward pass batch (as whole or one by one, I do it one by one)
 * <p style='color:blue'>[First batch forward pass]</p>
 * Calculate loss from forward pass result.
 * Average all losses gathered from batch forward pass.
 * After all done, do a backpropagation.
 * <p style='color:blue'>[First batch backpropagation]</p>
 * Take the __last__ sample from batch.
 * Use it to calculate gradients and deltas.
 * <p style='color:blue'>[Update weights]</p>
 * <p style='color:blue'>[Next batch]</p>
 * ...
 * <p style='color:blue'>[Last batch update]</p>
 * Sum all average batch losses and use it as loss of epoch
 * <p style='color:blue'>[Next epoch]</p>
 
This is the method we are told to use. Of course, after a little google search I found out that it is not how you do mini-batch. The way I see, mini batch is not used for to get average loss for batch. It is used to get average loss gradient for the batch. You do not update weights by single value. That is somewhat twisted version of stochastic learning. Batch, for example, require you to calculate gradients for every sample you feed into network but instead of using them to update you store their sum then at the end of batch you use that sum to update you weights. Only difference in mini batch is you do it for smaller sets which size determined beforehand. But since this is YOUR assignment, I will be implementing it your way.<br>
But it does not mean it is faulty method I am using causing such low performance because I tried the correct way too. Results did not improved, best I can get from validation still can't go past 35% boundary. <br>
As for batch, overall it is not suggested to use on some model as written <a href="http://axon.cs.byu.edu/papers/Wilson.nn03.batch.pdf">here</a>. Since our model obviously converges to a local minima, batch is not really helping.

#### <center>Analysis of dropout</center>

|Parameter|Value|
|-|-|
|Hidden activation|Sigmoid|
|Hidden layer count|1|
|Hidden layer node count|40|
|Output activation|Softmax|
|Loss function|Cross entropy|
|Epochs|300|
|Batch size|4|
|Learning rate bound|0.02 - 0.0005|
|Decay rate|0.0005|
|Momentum parameter|1.0|

- __Dropout probability = 0.0:__
> ![dropout_0.0](plots/dropout/0.0.png)

- __Dropout probability = 0.2:__
> ![dropout_0.2](plots/dropout/0.2.png)

- __Dropout probability = 0.5:__
> ![dropout_0.5](plots/dropout/0.5.png)

- __Dropout probability = 0.9:__
> ![dropout_0.9](plots/dropout/0.9.png)

> From the graphs above:
 - __From loss graphs__ we see that dropout affects loss negatively. Sadly these loss values are incorrect. The loss calculated includes the nodes that dropped out. That naturally drops the loss values. So I can't exactly use loss values for which value of dropout is best.
 - __From accuracy graphs__ we can, again, use training accuracy, as it is also includes nodes that dropped out. You can see this because as the dropout probability increses training accuracies starts do decline even more. But validation accuracy calculated without dropout so it is our only parameter for this analysis. 
 
> From these we can see that no dropout still gives us best result. That can be caused by many things:
 * Implementation error. Like most of the features of the code, dropout implemented later on. I did not do extensive testing but I am pretty sure it is working. Nonetheless there might some bug I might have missed. For example; I have a sneaking suspicion that I might have added dropout at wrong time. But my testings shows otherwise.
 * <a href="https://knowyourmeme.com/memes/rngesus" title="do you even pray?">RNG.</a> Since dropout has some randomized features result may change each run. Perhaps I did not choose the right parameters or masl I used to turn nodes off choose same nodes over and over. Super small detail (more like nitpicking) but possible.
 * I have to choose carefully where to apply dropout. Nodes connecting to output layer, <a href="https://stats.stackexchange.com/a/299305">as far as I heard</a>, is a bad place to put dropouts. That's why I did not added dropout to the last layer. But to see what kind of difference it does make, here is the graph of training results where __dropout probability is 0.5__ and output layer has dropout:
 ![fail](plots/dropout/0.5_output_dropout.png)
 Yep, it was a bad idea. <br>
 So what it did, it grew weights so much the become nan. So results became nan. That is probably because at some point all of the nodes of the output layer dropped out and softmax requires a division by their sum. That is essentially a zero division error but numpy does not really care and shows either nan or inf. <br>
 So we better leave output layer alone.
 
 So, what did we learn here? There is a chance that our model does not have depth enough to use regularization. Since my model probably converges local minima pretty quickly without overfitting, most of the time regularization does not help to improve my validation accuracy. But that does not mean I can't overfit the model:
 ![overfit-example](plots/overfit-redefined.png)
 Because, I can. It is just not a everyday occurrence.

#### <center>Analysis of momentum</center>

|Parameter|Value|
|-|-|
|Hidden activation|Sigmoid|
|Hidden layer count|1|
|Hidden layer node count|40|
|Output activation|Softmax|
|Loss function|Cross entropy|
|Epochs|300|
|Batch size|4|
|Learning rate bound|0.02 - 0.0005|
|Decay rate|0.0005|
|Dropout probability|0.0|

- __Momentum parameter = 1.0:__
> ![momentum_1.0](plots/dropout/0.0.png)

- __Momentum parameter = 0.8:__
> ![momentum_1.0](plots/momentum/0.8.png)

- __Momentum parameter = 0.5:__
> ![momentum_1.0](plots/momentum/0.5.png)

- __Momentum parameter = 0.2:__
> ![momentum_1.0](plots/momentum/0.2.png)

- __Momentum parameter = 0.0:__
> ![momentum_1.0](plots/momentum/0.0.png)

> I swear they are seperate graphs and I did not copy paste same file. Anyway, I implemented momentum by saving previous deltas for each layer then adding it on next update by multiplying it with $$ 1 - momentum\_parameter $$
But as you can see, there is nothing to analyze. It has such little impact on model we can say it is nonexistant. Perhaps I messed up this one too but it is unlikely since there is little to no calculation involved. <br>
Moving on.

#### <center>Analysis of learning rate</center>

|Parameter|Value|
|-|-|
|Hidden activation|Sigmoid|
|Hidden layer count|1|
|Hidden layer node count|40|
|Output activation|Softmax|
|Loss function|Cross entropy|
|Epochs|300|
|Batch size|4|
|Momentum parameter|1.0|
|Dropout probability|0.0|

This will be comparison of decaying learning rate vs. constant learning rate.

- __Decaying learning rate:__
> I used learning rate decay (as suggested to reduce local minima converge) but the method I use use constant decay instead of fancy algorithms like <a href="https://www.coursera.org/lecture/deep-neural-network/adam-optimization-algorithm-w9VCZ" title="I could have but the assignment dragged out as it is">Adam</a>. The learning rate would decay at some rate you gave to it at each epoch. Down below is the graph model with same parameters I always use. $$ learning\_rate\_bound = [0.0005, 0.02] \\ learning\_rate\_decay = 0.005 $$
> ![decay_lrate](plots/learning_rates/decay_0.0005.png)
Why decay? Decay is important because you want to have larger steps initally so that you can:
 - Converge faster initially
 - Step over small fluctuation in data so your model does not converge to some local minima.<br>
But as epochs increase it gotta get smaller so it does not overshoot global minima and converge to another local minima. So selecting an appropriate decay rate is also important. First you need to choose you decay rate based on your learning rate bounds. Also you do not want your learning rate to decay to minimum too fast or too slow. You want to have a value in between.<br>
Bounds are also important. You don't want to seperation to be too large or too small.

- __Constant learning rate:__
> Constant learning rate has it's pros and cons. For example: <br>
  __PROS__:
   - For large learning rates model converges more quickly and has less chance to converge to a local minima
   - For small learning rates model converges more accurately and with less fluctuations.
  __CONS__:
   - For large learning rates model might miss global minima and converge to another local minima after global minima. Also fluctuates a lot since changes have more impact.
   - For smaller learning rates model has high chance to converge to local minima. Also converges too slowly.
   
> For example:
 - __Learning rate = 0.0005:__
  ![constant_lr_0.0005](plots/learning_rates/constant_0.0005.png)
  You can see it is convergest slowly and without any hint of overfit
  
 - __Learning rate = 0.02:__
  ![constant_lr_0.0005](plots/learning_rates/constant_0.02.png)
  A lot of fluctuation. But sharper/faster converging on training loss and accuracy. But little to no change on validation accuracies.
  
 > From these data:
  - __Looking at loss values__ we can see that there is decay has it's definite benefits. Loss starts from much lower values, just like the constant high learning rate one, and quickly converged to lower values, even lower than one with constant low learning rate one. Meaning starting high allowed us to step over some local minimas easily. 
  - __Looking at training accuracies__ constant low rate allows validation accuracies to keep up with meaning less overfit. But our epoch rate is kinda low so we can't see big picture from graphs above. From the current graphs alone, best one is constant low learning rate one.
  - __Looking at validation accuracies__ it's hard to choose. Constant low learning rate shows amazing synchronicity on training and validation accuracies. But decay has such a smooth learning curve with promise of improvement on validation. I believe if we were to set epoch count much higher both graphs would more or less show same results. Decay just shows a somewhat improved result much faster.
  
Constant learning decay, of course, not the best kind of learning rate optimization. But it is nonetheless an improvement. Decay combines the advantages of both high and low learning rates into a small package. 

#### <center>Analysis of best parameters</center>

From all previous analysis I created pretrained models using best parameters. Below are these parameters and the resulting graph it shows:

|Parameter|Value|
|-|-|
|Hidden activation|Sigmoid|
|Hidden layer count|1|
|Hidden layer node count|40|
|Output activation|Softmax|
|Loss function|Cross entropy|
|Epochs|300|
|Batch size|4|
|Learning rate bound|0.02 - 0.0005|
|Decay rate|0.0005|
|Momentum parameter|1.0|
|Dropout probability|0.0|

 - __Single layer:__
  ![best_single_layer](plots/best_parameter/single_layer.png)
  
 - __Multi layer__
  ![best_multi_layer](plots/best_parameter/multi_layer.png)
  
Not the best by each parameter but best overall. 

### Visualization results

I visualized the weights that connects to output layer. Only works in single layer because, well, that's what is asked from us.

![visualization_graph](plots/visualization/single_layer.png)

I have no idea what this indicates. I was expecting a flower to appear but since accuracy is low it might prevent it to be seen. (At least I tried)

### Overall result

The result is, in my opinion, terrible. There is an issue of local minima convergence and nothing I tried fixed it. <br> Initially my model did not learn, at all. I tried shuffling data after each epoch and that improve my results. I reached to 30-35% max accuracy on validation dataset. After that it stuck like that. So far I tried these to improve my validation accuracies using;
 - L1, L2, dropout regularization. Nothing changed.
 - Momentum. Nothing changed.
 - Different activations. Nothing changed.
 - Different implementation for activation functions. Nothing changed.
 - Different loss functions and implementations. Nothing changed.
 - Adding/removing activation function before softmax. NOTHING CHANGED.
 - Different batching methods. Nothing changed.
 - Different backpropagation approach. Nothing changed.
 - Full object oriented solution. Got worse.
 - Combining training and validation dataset. Training on it then validating with test dataset. Got worse.<br>
You get the idea.
 
There might be many reasons for this _overall failure_:
 - I messed up big time during implementation of math functions (Loss and activation functions).
 - I failed to implement backpropagation algorithm.
 - Data is not suitable for such operations. (Using images as vectors to classify them? Sounds dangerous.)
 - I missed something vital(adam?). Then so does the others and libraries like keras(will come to that.)
 
Whatever the reason might be, there is some performance issue. I did not tried model for some other dataset or did not parse the images to matrix again since lack of time (3 weeks might be long for you. With 2 projects and an assignment it was long for me too) Keep in mind, I have written such long analysis because my code failed. I have to compensate using report. 

### Keras Results

In order to see whether data is the cause of the problems I am having I used a already verified neural network library, namely Keras.

|Parameter|Value|
|-|-|
|Hidden activation|Sigmoid|
|Hidden layer count|1|
|Hidden layer node count|40|
|Output activation|Softmax|
|Loss function|Cross entropy|
|Epochs|300|
|Batch size|16|
|Learning rate optimization|Adam|
|Dropout probability|0.5|

- __Single layer:__
    ![keras_single_layer](plots/keras/single_layer_no_dropout_300.png)
    
- __Multi layer:__
    ![keras_multi_layer](plots/keras/1_hidden_40_dropout_0.5_300.png)
    ![keras_multi_layer_score](plots/keras/keras_score_1_hidden_dropout_300.png)
    
    <center>Don't mind the cat. She likes to wander around when I am working.</center>
    
> - Using the graphs above let's analyze results:
    * Results are pretty similar to what we have on our model. Sometimes very high training accuracy with validation/test accuracy lower that 35%.
    * Loss graph is much sharper. But shape is comparable.
    
So nothing new or different than what we have. That indicates that our model, perhaps, is not that much of faulty after all. <br>
Or I messed up again. Not unlikely.

In [ ]:
""" This is the keras model I used to train the data. Please check if it's missing something. """

# define model
model = Sequential()
model.add(Dense(40, input_dim=768, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))


# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

![end](plots/end.jpeg)